# IS ZOC/MCM ROBUST AGAINST CORRUPTIONS
# DO METRICS STILL HOLD

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
# Configure Matplotlib to use LaTeX for text rendering
plt.rcParams['text.usetex'] = True

# Set fonts for Matplotlib
plt.rcParams['font.serif'] = 'Palatino'
plt.rcParams['font.sans-serif'] = 'Helvetica'
plt.rcParams['font.monospace'] = 'Courier'
plt.rcParams['font.size'] = 11
width_cm = 15
height_cm = 8
# Convert centimeters to inches
width_in = width_cm / 2.54
height_in = height_cm / 2.54
import seaborn as sns
sns.set_theme()
figsize= (width_in, height_in)


import os
from metrics.plotting_utils import save_plot, get_history_from_project, save_csv, get_dataset_name_mapping
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import wandb
api = wandb.Api()
from ood_detection.config import Config
from datasets.config import DATASETS_DICT
dataset_mapping = get_dataset_name_mapping()

## 1. Show influence of corruptions on metrics. three small plots

In [ ]:
# base metrics
clp = get_history_from_project('thesis-clp-temperature-100runs')
mmd = get_history_from_project('thesis-mmd-100')
zsa_class = tip_df = get_history_from_project('thesis-tip-adapters-16_shots-test')['ZEROSHOT']
clp_one_temp = clp[(clp['temperature'] < 1.13) & (clp['temperature'] > 0.98)]
clp_one_temp = clp_one_temp.drop(['temperature', 'std'], axis=1)
mmd = mmd.drop('std', axis=1)
distances = pd.concat([mmd, clp_one_temp, zsa_class], axis=1).rename(columns={'ZEROSHOT': 'zsa'})

In [ ]:
base = distances.copy()
base

In [ ]:
corr_orig = get_history_from_project('thesis-corruption-distances-final', set_name_index=False)


In [ ]:
corr_distances = corr_orig.copy()
corr_distances['corruption'] = corr_distances['name'].apply(lambda x: x.split('_')[1])
corr_distances['comboname'] = corr_distances['name'] +  '_' + corr_distances['severity'].astype(str)
corr_distances['name'] = corr_distances['name'].apply(lambda x: x.split('_')[0])
corr_distances = corr_distances.set_index('comboname').drop(['clp_std', 'mmd_std'], axis=1)
for name in corr_distances.name.unique():
    for c in corr_distances.corruption.unique():
        new_values = {}
        index_name = f"{name}_{c}_0"
        new_values['severity']= 0
        new_values['mmd'] = base.loc[name]['mmd']
        new_values['zsa'] =base.loc[name]['zsa'] / 100
        new_values['clp'] = base.loc[name]['clp']
        new_values['name']= name
        new_values['corruption']= c
        corr_distances.loc[index_name]= new_values

In [ ]:
corr_distances[corr_distances['severity'] == 0]

In [ ]:
corr_distances.severity.unique()

In [ ]:
corr_distances.head(1)

### Express increase / decrease in percentual change from origin

In [ ]:
# group by name and corruption. take value from 4 and substract 0
from collections import defaultdict
diff_dict = defaultdict(list)
difficulty_metrics = ['zsa', 'mmd', 'clp']
for name, group in corr_distances.groupby(['name', 'corruption']):
    min_df = group[group['severity']==0]
    max_df = group[group['severity']==5]

    for metric in difficulty_metrics:
        diff_dict[metric].append(max_df[metric][0] - min_df[metric][0])

In [ ]:
def get_mean_std(dicti, metric):
    print(f"{metric}: mean:{np.mean(dicti[metric])}, std:{np.std(dicti[metric])}")

for metric in difficulty_metrics:
    get_mean_std(diff_dict, metric)

In [ ]:
# 10 reduziert sich um 4, das sind: 4/10= 40%

In [ ]:
for metric in difficulty_metrics:
    orig_met = np.abs(base.mean()[metric])
    if metric=='zsa':
        orig_met = orig_met / 100
    change = np.abs(np.mean(diff_dict[metric]))
    print(f"{metric}: orig: {orig_met} to: {orig_met -change}, change: {change} is: {change/orig_met}")


#### --> take flowers brightness (zsa decreases nice), caltech snow (mmd) and caltech gaussian (clp)


In [ ]:
mmd_groups = [('caltech101', 'Snow'), ('dtd', 'Gaussian'), ('flowers102', 'Brightness')]
clp_groups = [('flowers102', 'Brightness'), ('caltech cub', 'Gaussian'), ('caltech101', 'Snow')]
zsa_groups = [('caltech cub', 'Brightness'), ('flowers102', 'Gaussian'), ('dtd', 'Snow')]

In [ ]:
figsize

In [ ]:
plt.rcParams['font.size'] = 14

In [ ]:
metrix = 'zsa'
fig, ax = plt.subplots(figsize=(5,5))
for name, group in corr_distances[[metrix, 'corruption', 'severity','name']].groupby(['name', 'corruption']):
    if name in zsa_groups:
    # if name:
        g = group.sort_values('severity')
        label_name = f"{dataset_mapping[name[0]]} - {name[1]}"
        ax.plot(g['severity'], g[metrix], label=label_name)
        ax.set_xticks(g['severity'])
        ax.set_xticklabels([0,1,3,5])
ax.set_ylabel(metrix.upper(), fontsize=18)
ax.set_xlabel('Severity of Corruption', fontsize=18)
# ax.set_title('Corruptions - Zero-Shot Accuracy')
ax.legend(fontsize=14)
plt.tight_layout()
save_plot(fig, metrix + 'corrupted', '5_3', False)

In [ ]:
metrix= 'mmd'
fig, ax = plt.subplots(figsize=(5,5))
for name, group in corr_distances[[metrix, 'corruption', 'severity','name']].groupby(['name', 'corruption']):
    if name in mmd_groups:
    # if name:
        g = group.sort_values('severity')
        label_name = f"{dataset_mapping[name[0]]} - {name[1]}"
        ax.plot(g['severity'], g[metrix], label=label_name)
        ax.set_xticks(g['severity'])
        #ax.set_xticklabels([0,1,3,5])
ax.set_ylabel(metrix.upper(), fontsize=18)
ax.set_xlabel('Severity of Corruption', fontsize=18)
# ax.set_title('Corruptions - Maximum Mean Discrepancy')
ax.legend(fontsize=14)
plt.tight_layout()
save_plot(fig, metrix + 'corrupted', '5_3', False)

In [ ]:
metrix = 'clp'
fig, ax = plt.subplots(figsize=(5,5))
for name, group in corr_distances[[metrix, 'corruption', 'severity','name']].groupby(['name', 'corruption']):
    if name in clp_groups:
    #if name:
        g = group.sort_values('severity')
        label_name = f"{dataset_mapping[name[0]]} - {name[1]}"
        ax.plot(g['severity'], g[metrix], label=label_name)
        ax.set_xticks(g['severity'])
ax.set_ylabel(metrix.upper(), fontsize=18)
ax.set_xlabel('Severity of Corruption', fontsize=18)
# ax.set_title('Corruptions -Confusion Log Probability')
ax.legend(fontsize=14)
plt.tight_layout()
save_plot(fig, metrix + 'corrupted', '5_3', False)

In [ ]:
plt.rcParams['font.size'] = 11

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=figsize)
for i, metrix in enumerate(['zsa', 'mmd', 'clp']):
    ax = axs[i]
    for name, group in corr_distances[[metrix, 'corruption', 'severity','name']].groupby(['name', 'corruption']):
        if name in clp_groups:
        #if name:
            g = group.sort_values('severity')
            label_name = f"{dataset_mapping[name[0]]} - {name[1]}"
            ax.plot(g['severity'], g[metrix], label=label_name)
            ax.set_xticks(g['severity'])
    ax.set_ylabel(metrix.upper())
    ax.set_xlabel('Severity of Corruption')
    # ax.set_title('Corruptions -Confusion Log Probability')
    ax.legend(fontsize=7)
    
    
    
plt.tight_layout()
save_plot(fig, metrix + 'all_three_metrics', '5_3', False)

## 2. Show sample of performance descrease for 1 or 2 datasets. Look how crowded it gets

# show influence on zoc, linear base line and mcm (maybe values, tables)

## 2.1 Show influence with table

In [ ]:
og_zoc = get_history_from_project('thesis-corruptions-zoc-final-5', set_name_index=False)
corr_auc = og_zoc.copy()
corr_auc['severity'] = corr_auc['severity'].astype(int)
corr_auc['comboname'] = corr_auc['name'] + '_' + corr_auc['severity'].astype(str)
corr_auc['corruption'] = corr_auc['name'].apply(lambda x: x.split("_")[1])
corr_auc['name'] = corr_auc['name'].apply(lambda x: x.split("_")[0])
corr_auc = corr_auc.set_index('comboname')

In [ ]:
# add zoc baselin
# get zoc
columns={'auc-sum': 'ZOC', 'auc-sum-std':'ZOC-std'}
full_zoc_1 = get_history_from_project('thesis-zsoodd_10_runs-std')
working = full_zoc_1.copy()
metric_of_interest = ['auc-sum', 'auc-sum-std']
working_zoc = working[metric_of_interest]

In [ ]:
corr_auc.head(3)

In [ ]:
final_corr = corr_auc.copy()
final_corr = final_corr[[col for col in final_corr.columns if col in ['auc-sum','name', 'auc-sum-std', 'corruption', 'severity']]]
for name in final_corr.name.unique():
    for c in final_corr.corruption.unique():
        new_values = {}
        index_name = f"{name}_{c}_0"
        new_values['severity']= 0
        new_values['auc-sum'] = working_zoc.loc[name]['auc-sum']
        new_values['auc-sum-std'] = working_zoc.loc[name]['auc-sum-std']
        new_values['name']= name
        new_values['corruption']= c
        final_corr.loc[index_name]= new_values

## Plot to group & mean all names or corruptions, best to show a trend!

In [ ]:
# make it possible to create plots grouped by a corruption, a name, severity
def plot_grouper_meaner(grouper, df, metric='zoc'):
    
    
    if metric == 'zoc':
        auroc = 'auc-sum'
    else:
        auroc = 'auc-max'
    metric_of_interest = [auroc, auroc+ '-std']
    all_groups = ['corruption','name']
    all_groups.remove(grouper)
    sevs = [0,1,3,5]

    leftover=all_groups[0]
    fig, ax = plt.subplots()
    for name, group in df.groupby(grouper):
        # for corruption
        df = group[[auroc,'severity','name']].sort_values('severity')
        # for each severity sum up all corruption values
        means =[]
        for sev in sevs:
            means.append(df[df['severity']==sev][auroc].mean())
        if grouper== 'name':
            ax.plot(sevs, means, label=dataset_mapping[name])
        else:
            ax.plot(sevs, means, label=name)

    # hline
    ax.hlines(0.5, xmin=0, xmax=len(means)+1, colors='darkgrey', alpha=0.9, linestyles='--', lw=2, label='Uninformed Guesser')
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=7)
    plt.subplots_adjust(right=0.75)

    ax.set_xlabel('Severity of Corruption')
    ax.set_ylabel('mean AUROC')
    t = ax.set_xticks(sevs)
    plt.tight_layout()

    save_plot(fig, f'{metric}_grouped_{grouper}_mean_{leftover}', '5_3', False)


In [ ]:
plot_grouper_meaner('name', final_corr)

In [ ]:
plot_grouper_meaner('corruption', final_corr)

In [ ]:
for name, group in final_corr.groupby(['name', 'severity', 'corruption']):
    if name[0] == 'gtsrb':
        if name[2] == 'Brightness':
            print(group)
            print("\n\n")

In [ ]:
final_corr.corr()

## Create monstrum again

In [ ]:
full_corruption_zoc = pd.concat([corr_distances, final_corr[['auc-sum', 'auc-sum-std']]], axis=1)

In [ ]:
of_interest = ['zsa', 'severity','mmd', 'clp', 'auc-sum','auc-sum-std']
full_corruption_zoc.dropna()[of_interest].corr()['auc-sum'].round(3)

## See, where the method breaks (ZOC)

In [ ]:
full_corruption_zoc[full_corruption_zoc['name'] == 'gtsrb']

In [ ]:
threshold = .04
full_corruption_zoc.dropna()[(full_corruption_zoc['auc-sum'] < 0.5 + threshold) & (full_corruption_zoc['auc-sum'] > 0.5-threshold)][of_interest].sort_index()

## MLS: Funktioniert 

In [ ]:
df_mls = get_history_from_project('thesis-corruptions-mls-1', set_name_index=False)
mls_auc = df_mls.copy()
mls_auc['severity'] = mls_auc['severity'].astype(int)
mls_auc['comboname'] = mls_auc['name'] + '_' + mls_auc['severity'].astype(str)
mls_auc['corruption'] = mls_auc['name'].apply(lambda x: x.split("_")[1])
mls_auc['name'] = mls_auc['name'].apply(lambda x: x.split("_")[0])
mls_auc = mls_auc.set_index('comboname')
full_mls = get_history_from_project('thesis-near-mcm-mls-1.0')
working = full_mls.copy()
metric_of_interest = ['auc-max', 'auc-max-std']
working_mls = working[metric_of_interest]
final_mls = mls_auc.copy()
final_mls = final_mls[[col for col in final_mls.columns if col in ['auc-max','name', 'auc-max-std', 'corruption', 'severity']]]
for name in final_mls.name.unique():
    for c in final_mls.corruption.unique():
        new_values = {}
        index_name = f"{name}_{c}_0"
        new_values['severity']= 0
        new_values['auc-max'] = working_mls.loc[name]['auc-max']
        new_values['auc-max-std'] = working_mls.loc[name]['auc-max-std']
        new_values['name']= name
        new_values['corruption']= c
        final_mls.loc[index_name]= new_values

In [ ]:
for name, group in final_mls.groupby(['name', 'severity', 'corruption']):
    if name[0] == 'gtsrb':
        if name[2] == 'Brightness':
            print(group)
            print("\n\n")

In [ ]:
def plot_grouper_meaner_double(groupers, df, metric):
    fig, axs = plt.subplots(nrows=len(groupers), figsize=figsize, sharex=True)

    for i, grouper in enumerate(groupers):
        if metric == 'zoc':
            auroc = 'auc-sum'
        else:
            auroc = 'auc-max'
        metric_of_interest = [auroc, auroc+ '-std']
        all_groups = ['corruption','name']
        all_groups.remove(grouper)
        sevs = [0,1,3,5]

        leftover=all_groups[0]
        for name, group in df.groupby(grouper):
            interm_df = group[[auroc,'severity','name']].sort_values('severity')
            means =[]
            for sev in sevs:
                means.append(interm_df[interm_df['severity']==sev][auroc].mean())
            if i == 0:
                axs[i].plot(sevs, means, label=dataset_mapping[name])
            else:
                axs[i].plot(sevs, means, label=name)
        # set yticks
        axs[i].set_yticks([0.5, 0.7, 0.9])
            

        axs[i].hlines(0.5, xmin=0, xmax=len(means)+1, colors='darkgrey', alpha=0.9, linestyles='--', lw=2, label='Uninformed Guesser')
        axs[i].legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=7)

    fig.text(0.4, 0, 'Severity of Corruption', ha='center')
    fig.text(0.0, 0.5, 'mean AUROC', va='center', rotation='vertical')
    
    # set xticks
    plt.xticks([0, 1, 3, 5])

    # set yticks
    plt.yticks([0.5, 0.7, 0.9])
    
    plt.tight_layout()
    save_plot(fig, f'{metric}_grouped_{str(groupers)}_mean', '5_3', False)
    
    
plot_grouper_meaner_double(['name', 'corruption'], final_mls, 'mls')

In [ ]:
plot_grouper_meaner('name',final_mls, 'mls')

In [ ]:
plot_grouper_meaner('corruption',final_mls, 'mls')

In [ ]:
full_corruption_df = pd.concat([corr_distances, final_mls[['auc-max', 'auc-max-std']]], axis=1)
of_interest = ['zsa', 'severity','mmd', 'clp', 'auc-max','auc-max-std', 'name', 'corruption']

full_corruption_mls = pd.concat([corr_distances, final_mls[['auc-max', 'auc-max-std']]], axis=1)
threshold = .05
full_corruption_mls.dropna()[(full_corruption_mls['auc-max'] < 0.5 + threshold) & (full_corruption_mls['auc-max'] > 0.5-threshold)][of_interest]

In [ ]:
full_corruption_df[full_corruption_df['severity'] ==0].sort_values('auc-max')

## MSP

In [ ]:
df_msp = get_history_from_project('thesis-corruptions-msp-1', set_name_index=False)
msp_auc = df_msp.copy()
msp_auc['severity'] = msp_auc['severity'].astype(int)
msp_auc['comboname'] = msp_auc['name'] + '_' + msp_auc['severity'].astype(str)
msp_auc['corruption'] = msp_auc['name'].apply(lambda x: x.split("_")[1])
msp_auc['name'] = msp_auc['name'].apply(lambda x: x.split("_")[0])
msp_auc = msp_auc.set_index('comboname')
full_msp = get_history_from_project('thesis-near-mcm-msp-1.0')
working = full_msp.copy()
metric_of_interest = ['auc-max', 'auc-max-std']
working_msp = working[metric_of_interest]
final_msp = msp_auc.copy()
final_msp = final_msp[[col for col in final_msp.columns if col in ['auc-max','name', 'auc-max-std', 'corruption', 'severity']]]
for name in final_msp.name.unique():
    for c in final_msp.corruption.unique():
        new_values = {}
        index_name = f"{name}_{c}_0"
        new_values['severity']= 0
        new_values['auc-max'] = working_msp.loc[name]['auc-max']
        new_values['auc-max-std'] = working_msp.loc[name]['auc-max-std']
        new_values['name']= name
        new_values['corruption']= c
        final_msp.loc[index_name]= new_values

In [ ]:
plot_grouper_meaner('name',final_msp, 'msp')
plot_grouper_meaner('corruption',final_msp, 'msp')

In [ ]:
full_corruption_msp = pd.concat([corr_distances, final_msp[['auc-max', 'auc-max-std']]], axis=1)
of_interest = ['zsa', 'severity','mmd', 'clp', 'auc-max','auc-max-std', 'name', 'corruption']

full_corruption_finals_mls = pd.concat([corr_distances, final_mls[['auc-max', 'auc-max-std']]], axis=1)


In [ ]:
threshold = .03
print('MLS')
full_corruption_mls.dropna()[(full_corruption_mls['auc-max'] < 0.5 + threshold) & (full_corruption_mls['auc-max'] > 0.5-threshold)][of_interest].sort_index()

In [ ]:
print('MSP')
full_corruption_msp.dropna()[(full_corruption_msp['auc-max'] < 0.5 + threshold) & (full_corruption_msp['auc-max'] > 0.5-threshold)][of_interest].sort_index()

In [ ]:
final_corr

In [ ]:
pd.concat([full_corruption_msp.rename(columns={'auc-max':'msp', 'auc-max-std': 'msp-std'})[['msp', 'msp-std']], full_corruption_mls.rename(columns={'auc-max':'mls','auc-max-std': 'mls-std'})[['mls', 'mls-std']], ], axis=1).mean()

In [ ]:
mls_msp = pd.concat([full_corruption_msp.rename(columns={'auc-max':'msp', 'auc-max-std': 'msp-std'})[['msp', 'msp-std']], full_corruption_mls.rename(columns={'auc-max':'mls','auc-max-std': 'mls-std'})[['mls', 'mls-std','name']]], axis=1)

In [ ]:
mls_msp[['msp', 'mls','name']].groupby('name').corr()

## Some hard facts

In [ ]:
full_corruption_mls['auc-max'].mean()

In [ ]:
full_corruption_mls.groupby(['name', 'severity']).mean().round(3)

In [ ]:
full_corruption_mls.groupby('corruption').mean()

In [ ]:
full_corruption_mls[full_corruption_mls['auc-max'] < 0.6]

### Now: look, where it breaks. Then, look for MMD, CLP & ZSA there. Theeeen, see if we reached these states with other and what happens

hol dir dir 0.5er indexe from orig. Dann sortier um nach clp. Dann schau, wo die 5er landen.

In [ ]:
threshold = 0.05

In [ ]:
resetted_index = full_corruption_mls.reset_index()

In [ ]:
 resetted_index[(resetted_index['auc-max'] < 0.5+threshold ) & (resetted_index['auc-max'] > 0.5-threshold )].corr().round(2)

In [ ]:
resetted_index[(resetted_index['auc-max'] < 0.5+threshold ) & (resetted_index['auc-max'] > 0.5-threshold )]

In [ ]:
resetted_index.corr()['auc-max'].round(2)

In [ ]:
below_threshold = resetted_index[(resetted_index['auc-max'] < 0.5+threshold ) & (resetted_index['auc-max'] > 0.5-threshold )].index.values.astype(int)

In [ ]:
below_threshold # indices from 0.5er

In [ ]:
resort = full_corruption_mls.reset_index().sort_values('mmd', ascending=True).reset_index()
indice = []
for b in below_threshold:
    # such im resorteten nach dem alten index b. hol dir davon den neuen Index, der Position in MMD Sortierung zeigt
    rb = resort[resort['index'] == b].index.values.astype(int)[0]
    indice.append(rb)
resort.loc[indice].sort_index()

In [ ]:
resort = full_corruption_mls.reset_index().sort_values('zsa', ascending=True).reset_index()
indice = []
for b in below_threshold:
    rb = resort[resort['index'] == b].index.values.astype(int)[0]
    indice.append(rb)
resort.loc[indice].sort_index()

In [ ]:
resort = full_corruption_mls.reset_index().sort_values('clp', ascending=False).reset_index()
indice = []
for b in below_threshold:
    rb = resort[resort['index'] == b].index.values.astype(int)[0]
    indice.append(rb)
resort.loc[indice].sort_index()

## ZOC VS MLS

In [ ]:
def plot_errorbar(df, value_column):
    error_corr = .5 * df[value_column + '-std']
    lower = df[value_column] - error_corr
    upper = df[value_column] + error_corr
    if value_column=='auc-max':
        name= 'MCM'
    else:
        name='ZOC'
    # mean = df[value_column].mean().round(3)
    # std = u"\u00B1{})".format(df[value_column+'-std'].mean().round(2))
    # label = name + f' (\u03bc= {mean}' + std
    label = name
    ax.plot(df[value_column], label = label)
    ax.plot(lower, color='tab:blue', alpha=.1)
    ax.plot(upper, color='tab:blue', alpha=.1)
    ax.fill_between(range(len(df)), lower, upper, alpha=.3)

In [ ]:
zoc_mls = pd.concat([full_corruption_zoc[['auc-sum', 'auc-sum-std']], full_corruption_mls[['auc-max', 'auc-max-std', 'name', 'corruption','severity']]],axis=1).dropna().sort_index()

In [ ]:
fig, ax = plt.subplots(figsize=(20,4))
plot_errorbar(zoc_mls, 'auc-max')
plot_errorbar(zoc_mls, 'auc-sum')
ax.hlines(0.5, xmin=-.5, xmax=[len(zoc_mls)], colors='darkgrey', alpha=0.9, linestyles='--', lw=2, label='Uninformed Guesser')
ticks = [dataset_mapping[col.split("_")[0]] for col in zoc_mls.index]
cotick = [col.split("_")[1] for col in zoc_mls.index]
_ = ax.set_xticklabels([ticks[i]+" / "+cotick[i] for i in range(len(zoc_mls))], rotation=90, ha='right')
ax.legend(fontsize=7)
ax.set_ylabel('AUROC')
ax.set_xlabel('Combination')
xticks = ax.get_xticks()
print(xticks)
for xtick in xticks[::2]:
    ax.axvline(x=xtick+0.5, color='gray', linestyle='--', alpha=0.5)
xticks = [i + 0.25 for i in range(1, len(xticks), 3)]
_ = ax.set_xticks(xticks)
plt.tight_layout()
save_plot(fig, 'corruption_zoc_mcm', '5_3', False)

In [ ]:
zoc_mls[zoc_mls['auc-sum']<0.53][['auc-max', 'auc-sum', 'auc-sum-std'] ]

In [ ]:
zoc_mls.mean().round(3)

In [ ]:
zoc_mls.corr().round(3)

In [ ]:
zoc_mls[['auc-max', 'auc-sum']].mean().round(3)

In [ ]:
zoc_mls[['auc-max', 'auc-sum']].std().round(3)

In [ ]:
zoc_mls[['auc-max', 'auc-sum','name']].groupby('name').corr()

In [ ]:
zoc_mls[['auc-max', 'auc-sum','corruption']].groupby('corruption').corr()

In [ ]:
zoc_mls[['auc-max', 'auc-sum','severity']].groupby('severity').corr()